In [1]:
import random
import ipyparallel as ipp

##### Declaring the number of nodes

In [2]:
n = 6

##### Generating node UIDs

In [3]:
uids = random.sample(range(1, (n * 10) + 1), n)

print(f"Generated {n} UIDs from the range [1, {n * 10}]")
print("Min UID:", min(uids))
print("Max UID:", max(uids))

Generated 6 UIDs from the range [1, 60]
Min UID: 2
Max UID: 58


##### Connecting to the engines

In [4]:
cluster = ipp.Cluster.from_file()
rc = cluster.connect_client_sync()
rc.wait_for_engines(n)

print(f"Number of available engines: {len(rc.ids)}")

Number of available engines: 6


#### 'Hello, World!' message from nodes

In [5]:
def hello_world(params):
    uid = params[0]
    n = params[1]
    return f"Hello, World! UID: {uid}, Total Nodes: {n}"

In [6]:
params = zip(uids, [n] * n)

results = rc[:].map_sync(hello_world, params)
print("\n".join(results))

Hello, World! UID: 41, Total Nodes: 6
Hello, World! UID: 36, Total Nodes: 6
Hello, World! UID: 40, Total Nodes: 6
Hello, World! UID: 58, Total Nodes: 6
Hello, World! UID: 2, Total Nodes: 6
Hello, World! UID: 29, Total Nodes: 6


#### LCR algorithm

In [7]:
# TODO

#### HS algorithm

In [8]:
# TODO